In [47]:
from pyspark import SparkContext
from pyspark.sql import SQLContext, Row

sc.stop()
sc = SparkContext()
sqlContext = SQLContext(sc)
rdd = sc.textFile("temperature-readings-tiny.csv")
parts = rdd.map(lambda l: l.split(";"))
df = parts.map(lambda p: Row(year=p[1].split("-")[0], \
			month=p[1].split("-")[1], temp=float(p[3]),station=int(p[0]) ))
df = sqlContext.createDataFrame(df)
df.registerTempTable("precReadings")
rdd = sc.textFile("stations.csv")
parts = rdd.map(lambda l: l.split(";"))
dfs = parts.map(lambda p: Row(station=int(p[0]) ))
dfs = sqlContext.createDataFrame(dfs)
dfs.registerTempTable("stations")

dfavg = df.groupBy("year","month","station").avg("temp").withColumnRenamed("avg(temp)","temp")
dfavg = dfavg.join(dfs, "station", "inner")
dfavg = dfavg.groupBy("year","month").avg("temp").withColumnRenamed("avg(temp)","temp")

df1 = dfavg.filter(df.year.between(1950, 2014))
df2 = dfavg.filter(df.year.between(1950, 1980))
df2 = df2.groupBy("month").avg("temp").withColumnRenamed("avg(temp)","temp")

df1 = df1.sort(df1.year.desc(), df1.month.desc())
df2 = df2.sort(df2.month.desc())

ave1 = df1.rdd
ave2 = df2.rdd
final = ave1.collect()
final = [list(x) for x in final]  # change Row to list
ave2 = dict(ave2.collect())

for i in range(len(final)):
    month = final[i][1]
    final[i][2] -= ave2[month]
    
lll = sc.parallelize(final)   # list to rdd
sqlContext = SQLContext(sc)   # rdd  to DF
df = sqlContext.createDataFrame(lll) 
df = df.sort(df._1.desc(), df._2.desc()).withColumnRenamed("_1","year") \
										.withColumnRenamed("_2","month")\
										.withColumnRenamed("_3","difference")
df.show()
# fff = df.rdd   # DF to rdd
# fff.saveAsTextFile("./results2/2q6") 


+----+-----+-------------------+
|year|month|         difference|
+----+-----+-------------------+
|2014|   12| -1.838561940815072|
|2014|   11| 2.9202591036414574|
|2014|   10| 2.6506731357636166|
|2014|   09|-0.9561130563336455|
|2014|   08|0.34340513857551613|
|2014|   07| 0.7707767430662145|
|2014|   06| 0.5867637800883099|
|2014|   05| 1.1918083405986657|
|2014|   04| 1.8507542360173952|
|2014|   03|  3.886612865295541|
|2014|   02| 3.4851983771565043|
|2014|   01| 3.1917645844887845|
|2013|   12|  4.672728381765573|
|2013|   11| 0.3435924369747903|
|1990|   07| 2.6110993237113753|
|1990|   02|   62.8101983771565|
|1970|   07| 16.111099323711375|
|1970|   02| 22.810198377156503|
|1961|   05| 1.0272486195873274|
|1961|   04|  1.581865347128507|
+----+-----+-------------------+
only showing top 20 rows



In [48]:
ave2

{'12': -3.9630509624107337,
 '11': -0.39525910364145694,
 '10': 4.45577847713961,
 '09': 9.541113056333645,
 '08': 13.525949700134163,
 '07': 18.888900676288625,
 '06': 13.856569553245023,
 '05': 9.564643272304563,
 '04': 2.925912430649273,
 '03': -1.989838671747154,
 '02': -2.8101983771565044,
 '01': -7.298216197392009}